In [1]:
import pickle
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="f053486a0d364fe09c5bddf49fc4657e",
                                                           client_secret="701cacd9264046d7889ab46f6f7610a9"))


In [2]:
f = open("Playlist_properties/playlist_ids.txt" ,"r")

playlist_list = []
for line in f:
    playlist_id, x, y = line.split() 
    playlist_list.append(playlist_id)
f.close()

f = open("Playlist_properties/playlist_names.txt" ,"r")
dict_ = open("Pickle_Files/Msd_id_to_spotify_id.pkl", "rb")
dict_ = pickle.load(dict_)
dict_ = dict(map(reversed, dict_.items()))
f.close()

print(len(playlist_list))



169


In [ ]:
main_dict = {}
for i in range(len(playlist_list)):
    playlist_id = playlist_list[i]
    offset = 0
    count = 0
    song_ids = set()
    while(1):
        results1 = sp.playlist_items(playlist_id, offset = offset, limit = 100)
        if(len(results1['items']) == 0):
            break
        for j in range(len(results1['items'])):
            try:
                song_id = results1['items'][j]['track']['id']
                if(song_id in dict_):
                    count+=1
                    song_ids.add(song_id)
            except:
                continue
        offset += len(results1['items'])
    song_ids = list(song_ids)
    
    for song_id in song_ids:
        if song_id not in main_dict:
            main_dict[song_id] = []
        main_dict[song_id].append(i)

# print(main_dict)


In [ ]:
print(len(main_dict))

In [ ]:
f = open("Pickle_Files/songs_to_playlist_mapping.pkl", "wb")
pickle.dump(main_dict, f)
f.close()

In [3]:
f = open("Pickle_Files/songs_to_playlist_mapping.pkl", "rb")
main_dict = pickle.load(f)
f.close()

In [4]:
import pandas as pd
df = pd.read_csv("Dataset/unlabelled_song_features_and_lda_topics.csv")

# to_skip = []

new_column = []
for i in df.index:
    try:
        song_id = df['sp_song_id'][i]
        new_column.append(main_dict[song_id])
    except:
        df.drop(i, inplace = True)
#         to_skip.append(i)
        print(i)
        continue

df["Playlists"] = new_column

new_column = []
for i in df.index:
#     if i in to_skip:
#         continue
    release_date = df['release_date'][i]
    try:
        release_date = int(release_date[-4:])
    except:
        release_date = int(release_date[:4])
    new_column.append(release_date)

df['release_date'] = new_column

df.to_csv("Dataset/updated_dataset.csv", index = False)

7958
9482


In [5]:
df.head()

,sp_song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,lda_topic_11,lda_topic_12,lda_topic_13,lda_topic_14,lda_topic_15,lda_topic_16,lda_topic_17,lda_topic_18,lda_topic_19,Playlists
0,6edQfeOlqbGteYixpJl3Sm,0.857,0.602,10,-8.311,1,0.0290,0.489000,0.02440,0.3170,...,0.0,0.022993,0.000000,0.122078,0.0,0.000000,0.000000,0.081108,0.386042,[165]
1,5Oe7wHPL4hdEXeF4AOayCi,0.322,0.990,8,-1.785,1,0.1710,0.000044,0.41700,0.0366,...,0.0,0.000000,0.163142,0.022999,0.0,0.144614,0.187207,0.000000,0.000000,[79]
2,6i1uWZYWabNHq2wQnoca58,0.666,0.884,9,-5.243,0,0.0429,0.506000,0.00612,0.0408,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.582131,[15]
3,4BzBtS6PBreni5hNPo2hos,0.609,0.578,9,-7.081,1,0.0414,0.296000,0.00000,0.1500,...,0.0,0.000000,0.511716,0.084905,0.0,0.000000,0.000000,0.115004,0.108140,[168]
4,0GvhHQbWSnGltjl0je61dI,0.699,0.514,4,-12.610,1,0.0315,0.587000,0.02900,0.2200,...,0.0,0.000000,0.000000,0.401954,0.0,0.000000,0.000000,0.000000,0.000000,"[30, 133]"
